# Simulation of EGF perturbations in drug-adapted BRAF<sup>V600E</sup> melanoma cells using the MARM1 model

Here you can simulate the time-course respose of drug-adapted BRAF<sup>V600E</sup> melanoma cells to addition of EGF. This code simulates the time-course response of melanoma cells adapted to a dose of RAF (vemurafenib) and MEK inhibitors (cobimetinib) and then stimulated with EGF.

**Note**:  multiple simulations of drug and ligand combinations for each of the 100 parameter sets are necessary to reproduce the totality of dose-response simulations presented in **Figures 5 and 6** of our work. We achieved such scalabilty by converting the model code in C and running it on multiple parallel cores for days, as explained in the Material & Methods. The code here is thus provided just as a way to explore MARM1 simulations. Substantial more infrastructure in terms of code optimization and parallel computing would be needed to reproduce the full dataset.  

## Import of libraries
Importing libraries necessary to run MARM1 model simulations.

In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Importing the MARM1 PySB model and the simulator.  

In [2]:
from pysb.simulator import ScipyOdeSimulator
from pysb.core import as_complex_pattern

from MARM1 import model

ValueError: An Expression can only be created from a sympy.Expr object

## User-defined experimental setup
In this section you can alter the setup of the experiment simulated by MARM1. First, you need to define the experimental setup of the pre-treatment phase, which consists in setting the EGFR expression level in the cell line and the concentration of the RAF and MEK inhibitor. The pre-treatment phase is simulated till a steady-state is reached. The variables needed for the pre-treatment phase are:

<b>EGFR_expression (fold-change from basal)</b>: defines the level of expression of the EGFR protein. A value of 1.0 represents the baseline expression in A375 cells as determined by absolute proteomics. Values > 1 and <1 denote overexpression and downregulation, respectively. We used a value of 10 and a value of 0.001 to simulate CRISPRa and CRISPRi control of EGFR expression. 

<b>RAFi_concentration ($\mu$M)</b>: defines the concentration of the RAF inhibitor (vemurafenib) during the pre-treatment and treatment phase. 

<b>MEKi_concentration ($\mu$M)</b>: defines the concentration of the RAF inhibitor (vemurafenib) during the pre-treatment and treatment phase. 

For the treatment phase, you need to set the concentration of the EGF ligand and the running time of simulation after ligand stimulation. This is done with the following variables:

<b>EGF_concentration (ng/mL)</b>: defines the EGF concentrations used to stimulate cells. We have calibrated the model only using data from 100 ng/mL, which is thus the default value.

<b>Simulation_time (h)</b>: define the simulation time after ligand stimulation. For the stimulation of cells expressing normal levels of EGFR, 2 hours is usually enough to visualize the pulsatile reactivation of MAPK signaling. To observe steady-state reactivation of MAPK signaling in EGFR overexpressing cells, 2 or more hours are usually necessary. In our work we used 8 hours for EGFR_expression=10 to match experimental data collected in CRISPRa EGFR cells.

Choose the experiment to simulate by setting the following variables and re-running the notebook:

1. **EGFR** under- or over-expression. 1.0 represents the typical copy number, < 1 denotes underexpression, and > 1 denotes overexpression.

In [ ]:
EGFR_expression = 1.0

2. **RAF inhibitor** pre-treatment concentration in μM.

In [ ]:
RAFi_concentration = 1.0

3. **MEK inhibitor** pre-treatment concentration in μM.

In [ ]:
MEKi_concentration = 0.0

4. **EGF** treatment concentration in ng/mL.

In [ ]:
EGF_concentration = 100

5. **Simulation time** in hours.

In [ ]:
t_trt = 2

6. (Optional) Which of the 100 best-fit parameter sets to use for the simulation. Set 0 is the best fit.

In [ ]:
param_set_index = 0

## Parameter set preparation

Read in the CSV file containing the best-fit parameter sets, and drop the chi$^2$ value which we don't need here.

In [ ]:
param_sets = pd.read_csv('parameter_sets.csv', index_col=0)
param_sets = param_sets.drop('chi2', axis=1)

Extract the chosen parameter set and convert it into a dict.

In [ ]:
params = param_sets.iloc[param_set_index].to_dict()

Override the `EGFR_crispr` parameter with the user-specified value for EGFR under/over-expression.

In [ ]:
params['EGFR_crispr'] = EGFR_expression

## Simulations

First we define some utility functions that will be used below.

In [ ]:
def equilibrate(simulator, initials):
    """Simulate a model from given initial conditions until it reaches steady state"""
    scale = 10
    t_start = 1e-4
    df = None
    tspan = np.geomspace(t_start, t_start * scale)
    while True:
        print(f"    at t={tspan[-1]:<5.3g} ... ", end='', flush=True)
        res = simulator.run(tspan=tspan, initials=initials)
        df = pd.concat([df, res.dataframe.iloc[1:]])
        initials = res.species[-1]
        close = np.isclose(
            *res.species[[-1,-2]].view(float).reshape(2,-1),
            rtol=1e-3
        )
        cs = np.sum(close)
        n = len(model.species)
        print(f"{cs}/{n} species converged")
        if np.all(close):
            break
        tspan *= scale
    return df

In [ ]:
def get_species_index(model, pattern):
    """Return the integer species number for a given species in the model"""
    pattern = as_complex_pattern(pattern)
    matches = [
        i for i, s in enumerate(model.species)
        if s.is_equivalent_to(pattern)
    ]
    n = len(matches)
    assert n == 1, f"Expected exactly one match, got {n}"
    return matches[0]

## Initial equilibrium
First we run the model from its baseline initial conditions until equilibrium is reached. For example protein synthesis/degradation, phosphorylation/dephosphorylation, and drug binding/unbinding all need to reach steady state to match the state of the cells in the experimental setup. There may be some time without visible progress as behind the scenes PySB runs BioNetGen to generate the reaction network and Cython to compile the resulting differential equations into efficient executable code.

In [ ]:
sim = ScipyOdeSimulator(model, param_values=params)
df_eq = equilibrate(sim, None)

Now that the model has been simulated once and the actual molecular species have been enumerated, we can find the exact species numbers for the inhibitors and EGF. These are needed so that their concentrations can be overridden in the model state for subsequent simulations.

In [ ]:
RAFi_index = get_species_index(model, model.monomers.RAFi(raf=None))
MEKi_index = get_species_index(model, model.monomers.MEKi(mek=None))
EGF_index = get_species_index(model, model.monomers.EGF(rtk=None))

## Inhibitor pre-treatment

In [ ]:
initials_pre = df_eq.iloc[-1, :len(model.species)].copy()
initials_pre[RAFi_index] = RAFi_concentration
initials_pre[MEKi_index] = MEKi_concentration
df_pre = equilibrate(sim, initials_pre)

We want to plot the first 24 hours of pre-treatment plus the state at final equilibrium, so we cut the time series down using a Pandas slice operation. Also we would like to compress the X-axis of this part of the plot, which we achieve by explicitly setting the index values (time in hours) to range from -0.5 to 0.

In [ ]:
df_pre = pd.concat([df_pre.loc[:24], df_pre.iloc[[-1]]])
df_pre = df_pre.set_index(np.linspace(-0.5, 0, len(df_pre)))
df_pre = df_pre.rename_axis(index='time')

## EGF treatment

In [ ]:
tspan_trt = np.linspace(0, t_trt, 1000)
initials_trt = df_pre.iloc[-1, :len(model.species)].copy()
initials_trt[EGF_index] = 100
res_trt = sim.run(tspan=tspan_trt, initials=initials_trt)
print()

In [ ]:
obs_names = [x.name for x in model.observables]
obs = pd.concat([df_pre, res_trt.dataframe])[obs_names]
obs.loc[:, (obs < 1e-10).all()] = 0

## Plot time courses

In [ ]:
obs_melted = obs.reset_index().melt(id_vars='time')
g = sns.FacetGrid(
    obs_melted, col='variable', hue='variable', col_wrap=5, sharey=False,
    size=1.5, aspect=1.25
)
g.map(plt.plot, 'time', 'value')
for ax in g.axes:
    _, ymax = ax.get_ylim()
    ax.set_ylim(ymin=0, ymax=ymax * 1.03)
    ax.set_xticks([-0.5, 0, 1, 2])
    ax.set_xticklabels(['-24', '0', '1', '2'])
    ax.axvline(0, color='k', linewidth=0.5, linestyle='--')
g.set_titles('{col_name}')
g.set_xlabels('time (hours)')
g.set_ylabels('amount (\u03bcM)')
plt.tight_layout()